<a href="https://colab.research.google.com/github/Vacayy/ai-playground/blob/main/finance_rag/Finance_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG 기반 해외 주식의 재무 데이터 분석
## 설명
- 분석하고자 하는 주식의 ticker를 입력합니다.
- yfinance를 기반으로 해당 주식의 재무 데이터(Income Statement (손익계산서), Balance Sheets (재무상태표), Cashflow (현금흐름표))를 불러옵니다.
  - JSON이 아닌 Pandas DataFrame 타입의 데이터입니다.
- 재무 데이터들을 전처리한 다음, context로 넣어줍니다.
- Model에 투자와 재무에 능통한 애널리스트 역할을 부여하여, 재무 건전성과 특이점을 분석합니다.
<br><br/>


> **아직 테스트 단계이므로, 단순하게 Microsoft 종목의 손익계산서로 한정하여 진행하였습니다.**



In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [6]:
# import bs4  # BeautifulSoup4: HTML 및 XML 파일을 파싱하기 위한 라이브러리
from langchain import hub  # LangChain의 다양한 유틸리티와 허브 사용
from langchain_chroma import Chroma  # 벡터 데이터 저장소인 Chroma 사용
from langchain_openai import ChatOpenAI  # OpenAI LLM 모델 사용
from langchain_openai import OpenAIEmbeddings  # OpenAI 임베딩 사용
# from langchain_community.document_loaders import WebBaseLoader  # 웹 문서를 불러오는 유틸리티
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 텍스트를 분할하는 도구

In [32]:
from google.colab import userdata

openai_api_key = userdata.get('openai_api_key')
llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key) # OpenAI 모델 초기화 (by langchain)

# [MY CODE] 재무 데이터 불러오기
- yfinance 를 이용하여 재무데이터를 불러옵니다.
- 재무데이터는 pandas의 dataframe 포맷입니다. 따라서 적절하게 전처리하는 과정이 필요합니다.

In [3]:
import yfinance as yf

msft = yf.Ticker("MSFT")
income_stmt = msft.income_stmt # 손익계산서

In [56]:
print(type(income_stmt))

<class 'pandas.core.frame.DataFrame'>


# [MY CODE] 데이터 전처리
- DataFrame 을 JSON으로 변환하고, 최종적으로 document 포맷으로 변환합니다.
- 단순히 JSON으로 변환하고 document 포맷 변환을 시도하니, 타입 에러가 발생했습니다.
- 따라서 JSON으로 변환하는 과정에서 모든 내부 key, value 값을 string으로 바꿔주었습니다.

In [30]:
from langchain.schema import Document
import json

# DataFrame을 JSON으로 변환
income_json = income_stmt.to_dict()

"""
document 로 변환하는 과정에서 str, int, float, bool, None 타입이 아니라면 에러가 발생합니다.
따라서 내부 모든 값을 string으로 변환합니다.
"""
# 키와 값의 Timestamp를 문자열로 변환
income_json_cleaned = {}
for key, value in income_json.items():
    key_str = str(key)
    # 내부 값도 dict 형태일 경우, 중첩된 값도 처리
    if isinstance(value, dict):
        value = {str(inner_key): inner_value for inner_key, inner_value in value.items()}
    income_json_cleaned[key_str] = value

# JSON 데이터를 Document로 변환
doc = Document(metadata={"source": "Microsoft Income Statement"}, page_content=json.dumps(income_json_cleaned, indent=2))

print(f'type(doc): {type(doc)}, | type(doc.page_content): {type(doc.page_content)}')
print('======================= ======================= =======================')
print(doc.page_content[:500])  # 일부 출력 테스트


type(doc): <class 'langchain_core.documents.base.Document'>, | type(doc.page_content): <class 'str'>
======================= ======================= =======================
{
  "2024-06-30 00:00:00": {
    "Tax Effect Of Unusual Items": -99918000.0,
    "Tax Rate For Calcs": 0.182,
    "Normalized EBITDA": 133558000000.0,
    "Total Unusual Items": -549000000.0,
    "Total Unusual Items Excluding Goodwill": -549000000.0,
    "Net Income From Continuing Operation Net Minority Interest": 88136000000.0,
    "Reconciled Depreciation": 22287000000.0,
    "Reconciled Cost Of Revenue": 74114000000.0,
    "EBITDA": 133009000000.0,
    "EBIT": 110722000000.0,
    "Net Inter


#[MY CODE] Document를 VectorDB에 저장하기

VectorDB에 넣기 전에 split을 진행하지 않았는데, 이유는 다음과 같습니다.
- 재무 상태표나 손익계산서 같은 데이터는 일반적으로 정해진 형식과 길이를 가지며, 그 길이가 아주 길지는 않습니다.
- 또한 각 데이터의 항목 간 상호 연관성이 있기 때문에 전체 맥락을 유지하는 것이 중요합니다.
- 따라서 이런 경우 split을 하지 않고 전체 데이터를 하나의 Document로 유지하는 것이 더 적합하다고 판단했습니다.

In [33]:
# 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    documents=[doc],  # 전체 데이터 하나의 Document로 처리
    embedding=OpenAIEmbeddings(api_key=openai_api_key)  # OpenAI 임베딩 사용
)

# 검색자 초기화
retriever = vectorstore.as_retriever()

# [MY CODE] 유저 질문 > 문서 검색

In [34]:
# 유저 질문
user_msg = "Microsoft의 손익계산서에서 기간별 순이익(Net Income) 추이를 요약해줘."

# 검색된 문서
retrieved_docs = retriever.invoke(user_msg)

# 검색된 문서 출력
for doc in retrieved_docs:
    print(doc.page_content)

{
  "2024-06-30 00:00:00": {
    "Tax Effect Of Unusual Items": -99918000.0,
    "Tax Rate For Calcs": 0.182,
    "Normalized EBITDA": 133558000000.0,
    "Total Unusual Items": -549000000.0,
    "Total Unusual Items Excluding Goodwill": -549000000.0,
    "Net Income From Continuing Operation Net Minority Interest": 88136000000.0,
    "Reconciled Depreciation": 22287000000.0,
    "Reconciled Cost Of Revenue": 74114000000.0,
    "EBITDA": 133009000000.0,
    "EBIT": 110722000000.0,
    "Net Interest Income": 222000000.0,
    "Interest Expense": 2935000000.0,
    "Interest Income": 3157000000.0,
    "Normalized Income": 88585082000.0,
    "Net Income From Continuing And Discontinued Operation": 88136000000.0,
    "Total Expenses": 135689000000.0,
    "Total Operating Income As Reported": 109433000000.0,
    "Diluted Average Shares": 7469000000.0,
    "Basic Average Shares": 7431000000.0,
    "Diluted EPS": 11.8,
    "Basic EPS": 11.86,
    "Diluted NI Availto Com Stockholders": 881360000

# [MY CODE] Prompt 생성
- rag-prompt, role_msg, user_msg 를 조합하여 prompt를 생성합니다.

In [62]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

prompt = hub.pull("rlm/rag-prompt") # 랭체인 허브에서 사전 정의된 RAG용 프롬프트를 가져옴

role_msg = 'You are a senior equity analyst with a deep knowledge of finance and investing. You provide factual, friendly, and detailed analysis, pointing out changes over time. When you need to compare numbers and do math, you do it by coding.'
user_prompt = prompt.invoke({"context": retrieved_docs[0].page_content, "role": role_msg, "question": user_msg})

messages = [
    SystemMessage(content=role_msg),  # role
    HumanMessage(content=user_prompt.to_string()) # 타입 에러로 인해 string 변환 후 넣어줌
]

print(messages[0])
print(messages[1].content[0:300])

content='You are a senior equity analyst with a deep knowledge of finance and investing. You provide factual, friendly, and detailed analysis, pointing out changes over time. When you need to compare numbers and do math, you do it by coding.' additional_kwargs={} response_metadata={}
Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: Microsoft의 최근 손익계산서에서 순이익(Net Income)을 요약해줘. 


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# [MY CODE] llm.invoke()
## 프롬프팅 전략

1. 역할 프롬프트를 따로 지정
2. 영어로 프롬프트 작성
3. 수치 계산은 코딩을 통해 해결하도록 하여 정확성 향상

As-Is: '당신은 시니어 주식 애널리스트입니다. 당신은 재무, 투자에 해박한 지식을 가지고 있으며, 시계열에 따른 변화를 예리하게 짚을 수 있습니다. 사실에 기반하면서도 자세한 분석을 제공합니다. 수치 비교 및 수학적 계산이 필요한 경우 반드시 코딩을 통해 수행하세요.'

> 응답: "*Microsoft의 2024년 6월 30일 기준 순이익은 88,136,000,000 달러입니다. 이는 전년도인 2023년 6월 30일의 72,361,000,000 달러와 비교하여 약 21.76% 증가한 수치입니다. 2022년과 비교하면 순이익은 7.74% 증가했습니다.*"

To-Be: You are a senior equity analyst with a deep knowledge of finance and investing. You provide factual, friendly, and detailed analysis, pointing out changes over time. When you need to compare numbers and do math, you do it by coding. ('당신은 재무, 투자에 해박한 지식을 가지고 있는 시니어 주식 애널리스트입니다. 시계열에 따른 변화를 짚어주며, 사실에 기반하면서도 친절하고 자세한 분석을 제공합니다. 수치 비교 및 수학적 계산이 필요한 경우 반드시 코딩을 통해 수행하세요.')

> 응답: Microsoft의 2024년 6월 30일 기준 순이익은 88,136,000,000달러이며, 이는 전년 동기인 2023년 6월 30일의 72,361,000,000달러에 비해 증가했습니다. 또한 2022년 6월 30일에는 72,738,000,000달러였으므로, 두 해를 비교했을 때 연속적인 성장세를 보였습니다. 이러한 성장 추세는 견고한 총 수익과 비용 관리의 결과로 해석될 수 있습니다.


In [53]:
response = llm(messages)
print(response.content)

Microsoft의 2024년 6월 30일 기준 순이익은 88,136,000,000달러이며, 이는 전년 동기인 2023년 6월 30일의 72,361,000,000달러에 비해 증가했습니다. 또한 2022년 6월 30일에는 72,738,000,000달러였으므로, 두 해를 비교했을 때 연속적인 성장세를 보였습니다. 이러한 성장 추세는 견고한 총 수익과 비용 관리의 결과로 해석될 수 있습니다.


# [MY CODE] 결과 분석
### 질문
> Microsoft의 손익계산서에서 기간별 순이익(Net Income) 추이를 요약해줘.

### 응답
> Microsoft의 2024년 6월 30일 기준 순이익은 88,136,000,000달러이며, 이는 전년 동기인 2023년 6월 30일의 72,361,000,000달러에 비해 증가했습니다. 또한 2022년 6월 30일에는 72,738,000,000달러였으므로, 두 해를 비교했을 때 연속적인 성장세를 보였습니다. 이러한 성장 추세는 견고한 총 수익과 비용 관리의 결과로 해석될 수 있습니다.

### 실제 순이익 데이터($)
- 2024-06-30: 88136000000.0
- 2023-06-30: 72361000000.0
- 2022-06-30: 72738000000.0
- 2021-06-30: 61271000000.0


실제 데이터에 기반한 분석을 내놓은 것을 확인할 수 있습니다.

In [55]:
net_income_data = income_stmt.loc["Net Income"]
print("=== 순이익 (Net Income) ===")
print(net_income_data)

=== 순이익 (Net Income) ===
2024-06-30    88136000000.0
2023-06-30    72361000000.0
2022-06-30    72738000000.0
2021-06-30    61271000000.0
Name: Net Income, dtype: object
